In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
model = ChatOpenAI(model='gpt-5-nano')

In [4]:
class Promptchain(TypedDict):
    title: str
    outline: str
    content: str

In [5]:
def create_outline(state: Promptchain) -> Promptchain:

    title = state['title']

    prompt = f"Generate a detailed outline for a blog on the topic: {title}"

    outline = model.invoke(prompt).content

    state['outline'] = outline

    return state

In [6]:
def create_blog(state: Promptchain) -> Promptchain:

    outline = state['outline']
    title = state['title']

    prompt = f'Write a detailed blog on the title - {title} using the follwing outline \n {outline}'

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [7]:
graph  = StateGraph(Promptchain)

graph.add_node('Create_Outline', create_outline)
graph.add_node('Create_Blog', create_blog)

graph.add_edge(START, 'Create_Outline')
graph.add_edge('Create_Outline', 'Create_Blog')
graph.add_edge('Create_Blog', END)

workflow = graph.compile()

In [8]:
initial_state = {'title' : 'The Future of Artificial Intelligence'}

final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'The Future of Artificial Intelligence', 'outline': 'Here is a detailed, ready-to-use outline you can adapt for a blog post or a multi-part series on The Future of Artificial Intelligence.\n\nTitle options (pick one or mix and match)\n- The Future of Artificial Intelligence: Opportunities, Risks, and What Comes Next\n- 2030 and Beyond: A Practical View of the Future of AI\n- AI’s Next Horizon: Technologies, Societies, and Strategies for the Next Decade\n- From Narrow AI to Broad Impact: How AI Will Shape Our World\n- The AI Era: Scenarios, Sectors, and the Shape of Society\n\nPurpose and audience\n- Purpose: Inform readers about likely trajectories in AI, what technologies and sectors to watch, and how individuals and organizations can prepare.\n- Audience: General readers with interest in technology; business leaders; policy makers; students and researchers; non-experts seeking a realistic, balanced view.\n\nVoice, scope, and framing\n- Tone: Clear, informed, balanced, with 